In [5]:
!python3 -V

Python 3.12.3


In [1]:
import pandas as pd

In [2]:
import pickle

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

Question 1. Downloading the data 

In [19]:
df = pd.read_parquet('../data/yellow_tripdata_2023-01.parquet')
print(len(df.columns))
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)



19


Question 2. Computing duration

In [20]:
print(df.duration.std())

42.59435124195458


Question 3. Dropping outliers

In [7]:
df.duration.describe(percentiles=[0.95,0.98,0.99])

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
50%      1.151667e+01
95%      3.646667e+01
98%      4.873333e+01
99%      5.725000e+01
max      1.002918e+04
Name: duration, dtype: float64

Question 4. One-hot encoding

In [16]:
categorical = ['PULocationID', 'DOLocationID']
df[categorical] = df[categorical].astype(str)
numerical = ['trip_distance']
train_dicts = df[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
print(X_train)

  (0, 44)	1.0
  (0, 326)	1.0
  (0, 518)	0.97
  (1, 149)	1.0
  (1, 458)	1.0
  (1, 518)	1.1
  (2, 150)	1.0
  (2, 463)	1.0
  (2, 518)	2.51
  (3, 228)	1.0
  (3, 300)	1.0
  (3, 518)	1.9
  (4, 238)	1.0
  (4, 267)	1.0
  (4, 518)	1.43
  (5, 39)	1.0
  (5, 326)	1.0
  (5, 518)	1.84
  (6, 46)	1.0
  (6, 411)	1.0
  (6, 518)	1.66
  (7, 109)	1.0
  (7, 305)	1.0
  (7, 518)	11.7
  (8, 148)	1.0
  :	:
  (3066757, 518)	9.44
  (3066758, 55)	1.0
  (3066758, 327)	1.0
  (3066758, 518)	0.0
  (3066759, 65)	1.0
  (3066759, 484)	1.0
  (3066759, 518)	0.0
  (3066760, 47)	1.0
  (3066760, 403)	1.0
  (3066760, 518)	0.82
  (3066761, 204)	1.0
  (3066761, 267)	1.0
  (3066761, 518)	3.05
  (3066762, 234)	1.0
  (3066762, 272)	1.0
  (3066762, 518)	5.8
  (3066763, 151)	1.0
  (3066763, 274)	1.0
  (3066763, 518)	4.67
  (3066764, 238)	1.0
  (3066764, 402)	1.0
  (3066764, 518)	3.15
  (3066765, 46)	1.0
  (3066765, 437)	1.0
  (3066765, 518)	2.85


In [10]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [11]:
df_train = read_dataframe('../data/yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('../data/yellow_tripdata_2023-02.parquet')

In [36]:
len(df_train), len(df_val)

(54373, 51497)

In [12]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [13]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

Question 6. Evaluating the model. RMSE on validation

In [14]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

/Users/romakaul/Desktop/Akshay/workspace/MLOps/.venv/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.820259863004852

Question 5. Training a model. RMSE on train

In [15]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

/Users/romakaul/Desktop/Akshay/workspace/MLOps/.venv/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.65839726314459